In [80]:
import cv2
import numpy as np
import os
import csv
path = "C:/Users/Sahith/Desktop/IEEE Sheet Vision/temp"
line = []
j = 0
for i in os.listdir('C:/Users/Sahith/Desktop/IEEE Sheet Vision/temp'):
    img = cv2.imread(path+'/'+i,0)
    print(i)
    #img = cv2.imread('C:/Users/Sahith/Desktop/IEEE Sheet Vision/ode_to_joy.jpg',0)
    height, width = img.shape[:2]
    #print(height,width)
    c1 = 540/height
    c2 = 960/width
    b = cv2.GaussianBlur(img,(3,3),0)
    """Resizing image"""
    img_res = cv2.resize(b,(int(1*width), int(1*height)))#, interpolation = cv.INTER_CUBIC)


    """Otsu Binarisation"""
    blur = cv2.GaussianBlur(img_res,(5,5),0)
    # if(c1 > 1.5 or c2 > 1.5):
    #     ret2,th2 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    #     print('ret2', ret2)

    ret2,th2 = cv2.threshold(img_res,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    #print('ret2', ret2)
    
    """Canny Edge Detection"""
    edges = cv2.Canny(th2,50,135,apertureSize = 3)
    #cv2.imshow('edges', edges)
    """Erosion - to get only the horizontal lines
        Subtract erosion from edges to remove the staff lines"""
    kernel = np.ones((1,5), np.uint8)

    erosion = cv2.erode(edges, kernel, iterations = 1)
    #cv2.imshow('erosion', erosion)

    result = cv2.subtract(edges, erosion)
    #cv2.imshow('result', result)

    """To find the y-values of staff lines"""
    sum_rows = np.sum(edges, axis=1)
    max_sum = np.amax(sum_rows)

    ind = np.where(sum_rows > 0.35 * max_sum)
    #print(ind)

    new_ind = ind[0][::2] + 1  ## contains y-values of staff lines

    ref_ind = new_ind[::5] ## Reference value for each set of 5 staff lines
    ref_ind2 = new_ind[4::5]

    bw = cv2.bitwise_not(th2)
    horizontal = bw.copy()
    vertical = bw.copy()

    cv2.waitKey(0)
    cv2.destroyAllWindows()

    #Staff line thickness
    from collections import Counter
    ind = ind[0][:]
    a = [x - ind[i-1] for i,x in enumerate(ind)][1:]
    data = Counter(a)
    b = data.most_common(1)
    slt = b[0][0] #slt is staff line thickness
    #slt

    #Removing the horizontal lines
    kernel_v = np.ones((2*slt,1),dtype = np.uint8)
    bw_erode1 = cv2.erode(vertical,kernel_v,iterations = 1)
    bw_dilate1 = cv2.dilate(bw_erode1,kernel_v,iterations = 1)
    #cv2.imshow('bw_dilate',bw_dilate1)
    cv2.waitKey(0)
    cv2.destroyAllWindows()  

    vertical_not = cv2.bitwise_not(bw_dilate1) #vertical
    #cv2.imshow('not',vertical_not)
    #cv2.imshow('bw_dilate',bw_dilate1)
    smooth = vertical_not.copy()
    blur1 = cv2.filter2D(smooth,-1,(np.ones((2,1),np.float32)/2))
    #cv2.imwrite('C:/Users/Sahith/Desktop/IEEE Sheet Vision/staff_removal.jpg',blur1)
    cv2.imshow('a',blur1)
    HU = cv2.HuMoments(cv2.moments(blur1)).flatten()
    row = [i,'Eighth-Note1']
    for l in (HU):
        row.append(l)
    line.append(row)
    print(line[j])                                          
    j += 1
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
with open('Test/Test.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    for row in line:
        writer.writerow(row)
writeFile.close()

Eighth.PNG
['Eighth.PNG', 'Eighth-Note1', 0.0007669027601391741, 4.265990311621262e-08, 1.407568734690487e-13, 1.9207481989083753e-13, -6.288928001771343e-27, 3.961451749326621e-17, -3.0949570502399203e-26]


In [6]:
import cv2
import numpy as np

img = cv2.imread('ode_to_joy.jpg',0)
height, width = img.shape[:2]

"""Resizing image"""
img_res = cv2.resize(img,(int(*width), int(1*height)))#, interpolation = cv.INTER_CUBIC)


#cv2.imshow('img_resize', img_res)
"""Otsu Binarisation"""
ret2,th2 = cv2.threshold(img_res,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
print('ret2', ret2)
#cv2.imshow('thresh', th2)

"""Canny Edge Detection"""
edges = cv2.Canny(th2,50,150,apertureSize = 3)

"""Erosion - to get only the horizontal lines
    Subtract erosion from edges to remove the staff lines"""
kernel = np.ones((1,5), np.uint8)

erosion = cv2.erode(edges, kernel, iterations = 1)
erosion_copy = erosion

cv2.imshow('erosion', erosion)

result = cv2.subtract(edges, erosion)
cv2.imshow('result', result)

"""To find the y-values of staff lines"""
sum_rows = np.sum(erosion, axis=1)
max_sum = np.amax(sum_rows)

ind = np.where(sum_rows > 0.7 * max_sum)
new_ind = ind[0][::2] + 1  ## contains y-values of staff lines

ref_ind = new_ind[::5] ## Reference value for each set of 5 staff lines
ref_ind2 = new_ind[4::5]
bw = cv2.bitwise_not(th2)
horizontal = bw.copy()
vertical = bw.copy()
cv2.waitKey(0)
cv2.destroyAllWindows()

    #Staff line thickness
from collections import Counter
ind = ind[0][:]
a = [x - ind[i-1] for i,x in enumerate(ind)][1:]
data = Counter(a)
b = data.most_common(1)
slt = b[0][0] #slt is staff line thickness
    #slt

    #Removing the horizontal lines
kernel_v = np.ones((2*slt,1),dtype = np.uint8)
bw_erode1 = cv2.erode(vertical,kernel_v,iterations = 1)
bw_dilate1 = cv2.dilate(bw_erode1,kernel_v,iterations = 1)
#cv2.imshow('bw_dilate',bw_dilate1)
 

vertical_not = cv2.bitwise_not(bw_dilate1) #vertical
#cv2.imshow('not',vertical_not)
#cv2.imshow('bw_dilate',bw_dilate1)
smooth = vertical_not.copy()
blur1 = cv2.filter2D(smooth,-1,(np.ones((2,1),np.float32)/2))
#cv2.imwrite('C:/Users/Sahith/Desktop/IEEE Sheet Vision/staff_removal.jpg',blur1)
cv2.imshow('a',blur1)
cv2.waitKey(0)
cv2.destroyAllWindows()    

####trying out some stuff

for i in ind[0]:
    erosion[i,:] = 255

cv2.imshow('erosion2', erosion)

result2 = cv2.add(result , erosion_copy)
result2 = cv2.subtract(result2, erosion)


kernel2 = np.ones((2,2), np.uint8)
result2 = cv2.dilate(result2,kernel2,iterations = 1)

# Run length encoding
count = np.zeros(960)

for i in range(55,125):
    for j in range(960):
        if(result2[i][j]==255):
            count[j]=count[j]+1

cv2.imshow('result2', result2)

count2 = np.zeros(225-125) 

for i in range(125,225):
    for j in range(624,659):
        if(result2[i][j]==255):
            count2[i-125] = count2[i-125] + 1


########################################



ref_ind = new_ind[::5] ## Reference value for each set of 5 staff lines
ref_ind2 = new_ind[4::5]

spacing = (ref_ind2 - ref_ind)/4 ## Spacing bw 2 staff lines
""""""

#cv2.imwrite('erosion.jpg', erosion)
#cv2.imwrite('edges.jpg', edges)
#cv2.imwrite('result.jpg', result)

min_line_len = 100
max_line_gap = 20

cv2.waitKey(0)
cv2.destroyAllWindows()

ret2 160.0


TypeError: 'numpy.int64' object is not iterable

In [71]:
import os
import shutil
import csv

path1 = "C:/Users/Sahith/Desktop/code/"
path_d = "C:/Users/Sahith/Desktop/code"

i = 0

with open('Final.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        if(len(row)==0):
            continue;
        if(row[1]=="F-Clif"):
            continue
        if not os.path.exists(path_d+'/'+"Train"):
            os.makedirs(path_d+'/'+"Train")
        if not os.path.exists(path1+"Train"+'/'+"Train.csv"):
                    with open(path1+"Train"+'/'+"Train.csv", 'w') as writeFile:
                            writer = csv.writer(writeFile)
                            writer.writerow(row)
                            writeFile.close()
        else:
             with open(path1+"Train"+'/'+"Train.csv", 'a') as writeFile:
                    writer = csv.writer(writeFile)
                    writer.writerow(row)
                    writeFile.close()
        
           
            

            
        
        
        #for item in row:
            #print(item)


In [81]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd 
from sklearn.neighbors import KNeighborsClassifier  


names = ['img','note','1','2','3','4','5','6','7']
dataset = pd.read_csv("Train/Train.csv", names=names)  
dataset1 = pd.read_csv("Test/Test.csv", names=names)  

x_train = dataset.iloc[:,2:].values
y_train = dataset.iloc[:,1].values
x_test = dataset1.iloc[:,2:].values
y_test = dataset1.iloc[:,1].values

from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(x_train)

x_train = scaler.transform(x_train)  
x_test = scaler.transform(x_test)  

classifier = KNeighborsClassifier(n_neighbors=1)  
classifier.fit(x_train, y_train)  

y_pred = classifier.predict(x_test)
print(y_pred)

['Eighth-Note']
